In [ ]:
input_path = "/opt/spark/data/jaffle_data"
bq_project = "lively-hall-447909-i7"
bq_dataset = "staging"
bq_table = "customers"
temp_gcp_bucket = "jaffle_shop_bronze_data"

In [ ]:
import os
gcp_keyfile = "../keys/gcp_keyfile.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=gcp_keyfile

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("CSV to BigQuery") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "com.google.cloud.spark:spark-bigquery-with-dependencies_2.12:0.28.0") \
    .config("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .config("spark.hadoop.google.cloud.auth.service.account.json.keyfile", gcp_keyfile) \
    .getOrCreate()

In [ ]:
df = spark.read.option("header", "true").csv(f"{input_path}/{bq_table}.csv")

In [ ]:
df.count()

In [10]:
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



In [ ]:
df.write \
        .format("bigquery") \
        .option("table", f"{bq_project}.{bq_dataset}.{bq_table}") \
        .option("writeMethod", "direct") \
        .mode("append") \
        .save()

In [11]:
spark.stop()